In [1]:
# %pip install domolibrary --upgrade
# %pip install domolibrary_execution --upgrade

## WARNING
-- as of 2/12/2025 - the test_full_auth workflow does not support instances MFA


In [2]:
import domojupyter as dj
import domolibrary_execution.utils.domojupyter as dxdj
import domolibrary.client.DomoAuth as dmda

config_auth = await dxdj.generate_domo_auth('domo-community', domojupyter_fn = dj)
# await config_auth.who_am_i()


🎉 full_auth token retrieved from domo-community ⚙️


In [3]:
import domolibrary.classes.DomoJupyter as dmdj

workspace_id = '70afbd1d-e85c-4846-8a0e-a46987011676'

async def get_workspace_access_token_accounts(workspace_id, auth):
    domo_workspace = await dmdj.DomoJupyterWorkspace.get_by_id(workspace_id, auth = auth)
        
    return [dja for dja in domo_workspace.account_configuration if dja.domo_account and dja.domo_account.data_provider_type == 'domo-access-token']
   
    
workspace_access_token_accounts = await get_workspace_access_token_accounts(workspace_id=workspace_id, auth = config_auth)
test_dja = workspace_access_token_accounts[-1]
# workspace_access_token_accounts[0:5]
test_dja

DomoJupyterAccount(account_id=146, alias='sdk_datacrew-space', is_exists=True, domo_account=DomoAccount_Credential(id=146, name='sdk_datacrew-space', data_provider_type='domo-access-token', created_dt=datetime.datetime(2025, 2, 11, 21, 47, 59), modified_dt=datetime.datetime(2025, 2, 11, 21, 49, 20), owners=None, is_admin_summary=False, target_instance=None, is_valid_full_auth=None, is_valid_token_auth=None, target_auth=None, target_user=None))

In [4]:
from typing import Callable
import domolibrary_execution.client.ExecutionError as ExecutionError

def extract_domo_instance_from_account_name(account_name):
    if not account_name.startswith('sdk_'):
        raise ExecutionError('account name does not start with sdk_')
    
    return account_name.split('_')[-1]
    

def populate_cred_config(dja : dmdj.DomoJupyterAccount,
                         domojupyter_fn : Callable
):
    dja.read_creds(domojupyter_fn = domojupyter_fn)
    
    dja.domo_account.target_instance = extract_domo_instance_from_account_name(dja.domo_account.name)
    
    return dja

populate_cred_config(dja = test_dja,
                     domojupyter_fn = dj)

DomoJupyterAccount(account_id=146, alias='sdk_datacrew-space', is_exists=True, domo_account=DomoAccount_Credential(id=146, name='sdk_datacrew-space', data_provider_type='domo-access-token', created_dt=datetime.datetime(2025, 2, 11, 21, 47, 59), modified_dt=datetime.datetime(2025, 2, 11, 21, 49, 20), owners=None, is_admin_summary=False, target_instance='datacrew-space', is_valid_full_auth=None, is_valid_token_auth=None, target_auth=None, target_user=None))

In [5]:
import domolibrary_extensions.utils.xkcd_password as xkcd
import domolibrary.client.DomoError as dmde

def generate_new_password():
    return xkcd.generate_domo_password(process_fn=xkcd.process_domo_password_fn, delimiter="-")

async def fix_password(
    domo_account,
    new_password: str = None,
    is_force_password_required: bool = False,

    backup_auth: dmda.DomoAuth = None,
    is_test_auth : bool = True,
    is_update_account: bool = True,
    
    debug_api: bool = False,
    debug_prn: bool = False,
    **kwargs,
):
    if is_test_auth:
        try:
            await domo_account.test_auths(debug_api = True)
        except dmde.DomoError as e:
            print(e)

    if domo_account.is_valid_full_auth:
        return f"No Action - {domo_account.name} username / password creds are valid"

    user_email = domo_account.Config.username

    if is_force_password_required and not user_email:
        raise dxde.ExecutionException(
            f"no username for {domo_account.id} - {domo_account.name}"
        )
    
    if not domo_account.target_auth:
        await domo_account.test_auths()
        
    if not domo_account.target_user:
        await domo_account.get_target_user()
    
    message = f"pasword reset - {domo_account.name} - {domo_account.target_user.display_name}"

    if debug_prn:
        print(message)

    await domo_account.update_target_user_password(
        new_password=new_password or generate_new_password(),
        is_test_auths=False,
        is_update_account=is_update_account,
        backup_auth=backup_auth,
        debug_api=debug_api,
    )
    
    return message

await fix_password(
    domo_account = test_dja.domo_account,
    debug_api = False,
    is_test_auth = False
)

🎉 token_auth token retrieved from datacrew-space ⚙️
🎉 full_auth token retrieved from datacrew-space ⚙️


'pasword reset - sdk_datacrew-space - sdk_datacrew-space'

In [6]:
import asyncio
from typing import List
import domolibrary.classes.DomoAccount as dmac
import domolibrary.classes.DomoAccessToken as dmat

async def fix_token(
    domo_account: dmac.DomoAccount_Credential,
    
    token_name : str = None,
    is_force_token_required : bool = False,
    token_update_threshold: int = 7 ,
    domo_access_tokens : List[dmat.DomoAccessToken] = None,
    
    backup_auth: dmda.DomoAuth = None,
    is_test_auth : bool = True,
    is_update_account: bool = True,
    debug_api: bool = False,
    debug_prn: bool = False,
    
    **kwargs,
):
    if is_test_auth:
        try:
            await domo_account.test_auths()
        except dmde.DomoError as e:
            print(e)
                     
    '''
    retrieve the domoAccessToken.
    cannot retrieve via target_user b/c not all access_token_accounts will have an email
    '''
    domo_access_tokens = domo_access_tokens or (
        await dmat.DomoAccessTokens(auth = domo_account.target_auth).get()
    )
    token_name = token_name or domo_account.name
    dat = next((dat for dat in domo_access_tokens if dat.name == token_name), None)
    
    
    # is valid and not expiring
    if domo_account.is_valid_token_auth and dat and dat.days_till_expiration > token_update_threshold:
        message = f'No Action - {dat.name} - valid token not expiring'
        if debug_prn:
            print(message)
        
        return message

    # dat (expiring or not valid)
    if dat:
        message = f'Regenerate - {dat.name} token - token expiring or invalid'
        
        if debug_prn:
            print(message)
            
        await dat.regenerate(debug_api = debug_api)
        return message
    
    # dat not required
    if not is_force_token_required and not domo_account.Config.domo_access_token:
        message = f"No Action {domo_account.name}- token_required is False"
        
        if debug_prn:
            print(message)
            
        return message 
    
    # no dat
    if not domo_account.target_user:
        await domo_account.get_target_user()
    
    message = f'Generate new access_token for {domo_account.name}'
    
    if debug_prn:
        print(message)
            
    await domo_account.reset_access_token(
        token_name = token_name,
        debug_api = debug_api,
        duration_in_days = 90
    )
    
    return message


await fix_token(
    domo_account = test_dja.domo_account,
)


🎉 token_auth token retrieved from datacrew-space ⚙️
🎉 full_auth token retrieved from datacrew-space ⚙️


'No Action - sdk_datacrew-space - valid token not expiring'

In [7]:

async def test_and_fix_domo_account(
    dja: dmdj.DomoJupyterAccount,
    backup_auth: dmda.DomoAuth = None,
    
    new_password: str = None,
    is_force_password_required: bool = False,

    token_name : str = None,
    is_force_token_required : bool = False,
    token_update_threshold: int = 7 ,
    domo_access_tokens : List[dmat.DomoAccessToken] = None,
    
    debug_api: bool = False,
    debug_prn: bool = False,

):
    domo_account = dja.domo_account

    await domo_account.test_auths()
    
    repair_functions = [fix_password, fix_token]

    for fn in repair_functions:
        await fn(
            domo_account=domo_account,
            backup_auth=backup_auth,
            is_test_auth = False,
            new_password = new_password,
            is_force_password_required=is_force_password_required,
            token_name = token_name,
            is_force_token_required = is_force_token_required,
            token_update_threshold = token_update_threshold,
            domo_access_tokens = domo_access_tokens,
            debug_api = debug_api,
            debug_prn = debug_prn
        )
    
    await domo_account.update_config(debug_api = debug_api)
    
    await asyncio.sleep(5)
    return await domo_account.test_auths()


await test_and_fix_domo_account(dja=test_dja, debug_prn = True)


🎉 token_auth token retrieved from datacrew-space ⚙️
🎉 full_auth token retrieved from datacrew-space ⚙️
No Action - sdk_datacrew-space - valid token not expiring
🎉 token_auth token retrieved from datacrew-space ⚙️
🎉 full_auth token retrieved from datacrew-space ⚙️


{'account_id': 146,
 'alias': 'sdk_datacrew-space',
 'target_instance': 'datacrew-space',
 'is_valid_full_auth': True,
 'is_valid_token_auth': True}

## Deploy Remote Instance


In [33]:
test_remote_auth = await dxdj.generate_token_auth('datacrew-space', domojupyter_fn = dj)

🎉 token_auth token retrieved from datacrew-space ⚙️


In [37]:
import domolibrary.classes.DomoGroup as dmdg
import domolibrary.classes.DomoUser as dmdu

async def generate_domo_group(auth : dmda.DomoAuth,
                              group_name : str,
                              owner_user_email_ls: List[str] = None,
                              owner_group_name_ls : List[str] = None,
                               debug_api : bool = False
                             ):
    
    domo_group = await dmdg.DomoGroups(auth = auth).upsert(
        group_name = group_name,
        group_type = dmdg.GroupType_Enum.CLOSED.value,
        description = "generated for managing account access",
        debug_api = debug_api,
    )

    group_owners = []
    domo_users = []
    
    if owner_user_email_ls:
        domo_users = await dmdu.DomoUsers(auth = auth).search_by_email(owner_user_email_ls, 
                                                                       only_allow_one = False,
                                                                       debug_api = debug_api)
        group_owners += domo_users
        
    if owner_group_name_ls:
        group_owners += await dmdg.DomoGroups(auth = auth).search_by_name(
            group_name = owner_group_name_ls,
            only_allow_one = False,
            debug_api = debug_api
        )
        
    await domo_group.Membership.add_owners(group_owners, debug_api = debug_api)
    
    await domo_group.Membership.add_members(domo_users, debug_api  = debug_api)
    
    return domo_group
    

async def deploy_account_to_remote_instance(remote_auth : dmda.DomoAuth, 
                                            account_name : str, 
                                            account_config : dmac.AccountConfig,
                                            debug_api : bool = False,
                                            share_default_group : dmdg.DomoGroup = None,
                                            owner_group_name_ls : List[str] = None,
                                            owner_user_email_ls : List[str] = None
) :
    
    
    remote_account = await dmac.DomoAccounts.upsert_account(auth = remote_auth,
                                                            account_name = account_name,
                                                            account_config = account_config,
                                                            debug_api = debug_api,
                                                           )
    
    share_default_group = await generate_domo_group(
        auth = remote_auth,
        group_name = account_name,
        owner_group_name_ls = owner_group_name_ls,
        owner_user_email_ls = owner_user_email_ls
    )
    
    await remote_account.Access.upsert_share(
        domo_group = share_default_group,
        debug_api = debug_api
    )
        
    return remote_account


await deploy_account_to_remote_instance(
    remote_auth = test_remote_auth,
    account_name = test_dja.domo_account.name,
    account_config = test_dja.domo_account.Config,
)

DomoAccount(id=10, name='sdk_datacrew-space', data_provider_type='domo-access-token', created_dt=datetime.datetime(2025, 2, 12, 22, 55, 37), modified_dt=datetime.datetime(2025, 2, 13, 0, 22, 45), owners=[DictDot(id='402376113', type='USER', displayName='sdk_datacrew-space')], is_admin_summary=True)

In [ ]:
test_deploy = [
    {
        "domo_instance" : "datacrew-space",
        "owner_group_name" : 
        
    }

In [ ]:
@dataclass
class Config:
    